In [2]:
# ! pip install nltk

In [4]:
import numpy as np
import pandas as pd
from siuba import *

from calitp_data_analysis.sql import to_snakecase

from shared_utils import geography_utils
import dla_utils

import utils

In [5]:
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_colwidth', None)

In [6]:
GCS_FILE_PATH  = 'gs://calitp-analytics-data/data-analyses/dla/dla-iija'

In [9]:
df = to_snakecase(pd.read_excel(f"{GCS_FILE_PATH}/AllOutcomesasof02-17-2023.xlsx"))

In [10]:
df

,dist_ppno,district,ppno,ea,project_id,funding_sources,on_system,outcome_desc,unit_desc,total,parent_ppnos,parent_ids,reporting_period,report_type
0,05-0057D,5,0057D,31601,0518000120,TCEP,Y,Air Quality & GHG (only ‘Change’ required): Carbon Dioxide (CO2),Tons,0,NaN,NaN,FY 20/21 Q4,PROGRESS
1,05-0057D,5,0057D,31601,0518000120,TCEP,Y,Air Quality & GHG (only ‘Change’ required): Carbon Dioxide (CO2),Tons,0,NaN,NaN,FY 21/22 Q1,PROGRESS
2,05-0057D,5,0057D,31601,0518000120,TCEP,Y,Air Quality & GHG (only ‘Change’ required): Carbon Dioxide (CO2),Tons,0,NaN,NaN,FY 21/22 Q2,PROGRESS
3,05-0057D,5,0057D,31601,0518000120,TCEP,Y,Air Quality & GHG (only ‘Change’ required): Carbon Dioxide (CO2),Tons,0,NaN,NaN,FY 21/22 Q3,PROGRESS
4,05-0057D,5,0057D,31601,0518000120,TCEP,Y,Air Quality & GHG (only ‘Change’ required): Carbon Dioxide (CO2),Tons,0,NaN,NaN,FY 21/22 Q4,PROGRESS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17691,75-T0019,75,T0019,NaN,0021000176,TCEP,N,Throughput: Change in Cargo Volume That Can Be Accommodated,# of Tons,96890613,NaN,NaN,FY 22/23 Q2,PROGRESS
17692,75-T0019,75,T0019,NaN,0021000176,TCEP,N,Throughput: Change in Rail Volume That Can Be Accommodated,# of Containers,269141,NaN,NaN,FY 22/23 Q2,PROGRESS
17693,75-T0019,75,T0019,NaN,0021000176,TCEP,N,Throughput: Change in Rail Volume That Can Be Accommodated,# of Trailers,1019,NaN,NaN,FY 22/23 Q2,PROGRESS
17694,75-T0019,75,T0019,NaN,0021000176,TCEP,N,Throughput: Change in Truck Volume That Can Be Accommodated,# of Trucks,1295,NaN,NaN,FY 22/23 Q2,PROGRESS
